In [3]:
import json
import pprint

with open('cities_usa_Scale8.topo.json') as f:
    d = json.load(f)
    print(d['objects']['citiesScale8']['geometries'][0]["properties"].keys())
#    pprint.pprint(d)

dict_keys(['SCALERANK', 'NATSCALE', 'LABELRANK', 'FEATURECLA', 'NAME', 'NAMEPAR', 'NAMEALT', 'DIFFASCII', 'NAMEASCII', 'ADM0CAP', 'CAPIN', 'WORLDCITY', 'MEGACITY', 'SOV0NAME', 'SOV_A3', 'ADM0NAME', 'ADM0_A3', 'ADM1NAME', 'ISO_A2', 'NOTE', 'LATITUDE', 'LONGITUDE', 'CHANGED', 'NAMEDIFF', 'DIFFNOTE', 'POP_MAX', 'POP_MIN', 'POP_OTHER', 'RANK_MAX', 'RANK_MIN', 'GEONAMEID', 'MEGANAME', 'LS_NAME', 'LS_MATCH', 'CHECKME', 'MAX_POP10', 'MAX_POP20', 'MAX_POP50', 'MAX_POP300', 'MAX_POP310', 'MAX_NATSCA', 'MIN_AREAKM', 'MAX_AREAKM', 'MIN_AREAMI', 'MAX_AREAMI', 'MIN_PERKM', 'MAX_PERKM', 'MIN_PERMI', 'MAX_PERMI', 'MIN_BBXMIN', 'MAX_BBXMIN', 'MIN_BBXMAX', 'MAX_BBXMAX', 'MIN_BBYMIN', 'MAX_BBYMIN', 'MIN_BBYMAX', 'MAX_BBYMAX', 'MEAN_BBXC', 'MEAN_BBYC', 'COMPARE', 'GN_ASCII', 'FEATURE_CL', 'FEATURE_CO', 'ADMIN1_COD', 'GN_POP', 'ELEVATION', 'GTOPO30', 'TIMEZONE', 'GEONAMESNO', 'UN_FID', 'UN_ADM0', 'UN_LAT', 'UN_LONG', 'POP1950', 'POP1955', 'POP1960', 'POP1965', 'POP1970', 'POP1975', 'POP1980', 'POP1985', '

In [12]:
# Manually change some states when misclassified
with open('US_States20190708.json') as f:
    d = json.load(f)
    #print(d)
    for dic in d:
        # substract 252 misclassifications for the city Norman, OK
        if dic["geo_adm1_code"] == "OK": dic["count"] -= 252
        # substract 76 misclassifications for the city Lincoln, NE
        if dic["geo_adm1_code"] == "NE": dic["count"] -= 76
        # substract 540 misclassifications for the city Provo, UT
        if dic["geo_adm1_code"] == "UT": dic["count"] -= 540
        # substract 186 misclassifications for the city Idaho, ID
        if dic["geo_adm1_code"] == "ID": dic["count"] -= 183
        # substract 163 misclassifications for the city Big Spring, TX
        if dic["geo_adm1_code"] == "OK": dic["count"] -= 163         
            
        print(dic["geo_adm1_code"], dic["count"]/dic["population"])
    #print(d)
    
    with open('US_States20190708_manuallyCorrectedSomeMisclassifications.json', 'w') as fi:
        json.dump(d, fi)

MS 0.0001878422304489843
NE 0.0007095713608577222
IA 0.0006074429528157265
MT 0.0005329333467999287
CO 0.0008066463900757273
ND 0.0005328858783656263
WI 0.0005073016753963023
FL 0.00042062191607118645
NH 0.00029326493523859306
MN 0.0007423249492945579
WY 0.00025894087253190804
HI 0.0006688885081995296
CA 0.0004921745545834873
VA 0.000514858003863463
ID 0.0004969771927000828
UT 0.0006183723746982293
CT 0.0004088171830228033
AR 0.00025782999973528
WV 0.0003438087741099341
NY 0.0008121719326579035
ME 0.0004632151355168319
LA 0.00040545277515927476
TN 0.0005349531659033826
KY 0.0004992779489852056
AL 0.00023199269807949336
AK 0.0008052882614098902
TX 0.0006223637679284764
OK 0.0008175810370818108
NC 0.000515945958405907
AZ 0.0006071139083827594
NM 0.0003926419628124667
VT 0.0003154518567624391
DC 0.007848915615106266
SC 0.00022317618483149015
IN 0.0008490355706005793
GA 0.0005545997801654709
RI 0.0003237194989583849
PA 0.0003506255837228704
NJ 0.0003576555893227123
KS 0.000502050709310815


In [6]:
with open('US_cities_States20190708.json', "r") as f:
    cities = json.load(f)
    
    with open('cities_usa_Scale8.topo.json') as g:
        topo = json.load(g)
        print("Number cities:", len(cities))
           
        for i in range(len(topo['objects']['citiesScale8']['geometries'])):
            if "North" in topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"]:
                print(topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"])
            # correct wrong geolocalised North Platte to Lincoln in Nebraska
            # And dont consider tweets of Lincoln because most misclassified
            if topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Lincoln" and topo['objects']['citiesScale8']['geometries'][i]["properties"]["ADM1NAME"] == "Nebraska":
                #print(topo['objects']['citiesScale8']['geometries'][i]["properties"])
                    
                for city in cities:
                    if city["city_name"] == "North Platte": 
                        print( city["count"],  city["population"])
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["count"] = city["count"]
                    if city["city_name"] == "Lincoln":
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["population"] = city["population"]
    
            
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Big Spring": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Twin Falls": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "North Platte": pass # partially wrong
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Norman": pass # partially wrong; Oklahoma
            elif topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"] == "Provo": pass # partially wrong
       
            
            else:
                for city in cities:    
                
                    # Check that New York is considered. Might be written: New York City
                    if city["city_name"] == topo['objects']['citiesScale8']['geometries'][i]["properties"]["NAME"]:

                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["count"] = city["count"]
                        topo['objects']['citiesScale8']['geometries'][i]["properties"]["population"] = city["population"]
            
        print("Write ..")    
        with open('cities_usa_Scale8_CountPopul_20190708.topo.json', 'w', encoding='utf-8') as wr:
            json.dump(topo, wr, ensure_ascii=False, indent=4)

Number cities: 5805
North Platte
193 24194
Write ..
